In [1]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time

import category_encoders as ce
import catboost
from sklearn import metrics

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric
#from messaging import send_message

pd.set_option('display.max_columns', None)

In [2]:
# metrics in catboost format
class AmexMetric:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):    
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = approxes[0]
        target = target
        return compute_amex_metric(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class RecallAt4:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                                   
        return compute_recall_at4(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class NormGini:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                   
        return compute_normalized_gini(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

In [3]:
# CONFIG PARAMS
N_REPEATS = 3

In [4]:
OOF_PATH = Path("../data/oof/catboost-dsv02")
SUB_PATH = Path("../data/subs/catboost-dsv02")
ART_PATH = Path("../artifacts/catboost-dsv02")

if not OOF_PATH.exists():
    OOF_PATH.mkdir(parents=True, exist_ok=True)
if not SUB_PATH.exists():
    SUB_PATH.mkdir(parents=True, exist_ok=True)
if not ART_PATH.exists():
    ART_PATH.mkdir(parents=True, exist_ok=True)

***
## load and prepare data

In [5]:
!ls ../data/processed/dsv02

test.parquet  train.parquet


In [6]:
train = pd.read_parquet("../data/processed/dsv02/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [7]:
input_feats = train.columns.tolist()
categ_feats = [
    'B_30_first', 'B_38_first', 'D_114_first', 'D_116_first', 'D_117_first', 
    'D_120_first', 'D_126_first', 'D_63_first', 'D_64_first', 'D_66_first', 'D_68_first',
    'B_30_last', 'B_38_last', 'D_114_last', 'D_116_last', 'D_117_last', 
    'D_120_last', 'D_126_last', 'D_63_last', 'D_64_last', 'D_66_last', 'D_68_last',
]
len(input_feats)

1562

In [8]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
del train_labels
gc.collect()

0

***
## model training

train with repeated cross validation

In [9]:
model_params = {
    #'eval_metric':'AUC',
    'eval_metric':AmexMetric(),
    'depth': 5,
    'learning_rate': 0.01,
    'iterations': 5000,
    'early_stopping_rounds': 300,
    'use_best_model':True,
    #'subsample': 0.9,
    'bootstrap_type':'Bayesian',
    'rsm': 0.3,
    'l2_leaf_reg': 1.,
    'min_data_in_leaf': 1000,
    'nan_mode':'Min',
    'random_seed': 2112,
}

In [10]:
def train_models(dataframe: pd.DataFrame, n_folds: int = 5,) -> tuple:
    
    models = list()
    
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for fold in range(n_folds):
        
        print(f" training model {fold+1}/{n_folds} ".center(100, "#"))
        
        train_df = dataframe.query("fold != @fold").copy()
        valid_df = dataframe.query("fold == @fold").copy()
                
        train_dset = catboost.Pool(
            data=train_df.loc[:,input_feats],
            label=train_df.loc[:,"target"].values,
            cat_features=categ_feats,
        )
        valid_dset = catboost.Pool(
            data=valid_df.loc[:,input_feats],
            label=valid_df.loc[:,"target"].values,
            cat_features=categ_feats,
        )
        
        model = catboost.CatBoostClassifier(**model_params)
        model.fit(
            train_dset,
            eval_set=valid_dset,
            verbose=20
        )
        
        #lgb.plot_importance(model, figsize=(8,15), importance_type="split", max_num_features=30)
        #lgb.plot_importance(model, figsize=(8,15), importance_type="gain", max_num_features=30)
        #plt.show()        
        
        oof.loc[valid_df.index,"pred"] = model.predict(valid_dset, prediction_type="Probability")[:,1]
        
        models.append(model)
        del train_df,valid_df,train_dset,valid_dset
        gc.collect()
    
    return models,oof

In [11]:
# implement repeated cross validation
sorted(glob("../data/processed/cv*.csv"))

['../data/processed/cv0.csv',
 '../data/processed/cv1.csv',
 '../data/processed/cv2.csv',
 '../data/processed/cv3.csv',
 '../data/processed/cv4.csv',
 '../data/processed/cv5.csv',
 '../data/processed/cv6.csv',
 '../data/processed/cv7.csv',
 '../data/processed/cv8.csv',
 '../data/processed/cv9.csv']

In [12]:
%%time 

all_models = list()
all_oof_dfs = list()

for repetition in range(N_REPEATS):
    print(f" repeated cross-validation step: {repetition+1}/{N_REPEATS} ".center(100, "#"))

    folds = pd.read_csv(f'../data/processed/cv{repetition}.csv', index_col="customer_ID")
    _train = pd.merge(train, folds, how="inner", left_index=True, right_index=True).reset_index(drop=True)
    
    tic = time.time()
    models,oof = train_models(_train)
    tac = time.time()
    print(f"Training time: {(tac-tic)/60} min.")
          
    # oof metrics
    print("OOF recall_at4:", compute_recall_at4(oof.target.values, oof.pred.values))
    print("OOF normalized_gini:", compute_normalized_gini(oof.target.values, oof.pred.values))
    print("OOF competition metric:", compute_amex_metric(oof.target.values, oof.pred.values))
    
    all_models.append(models)
    all_oof_dfs.append(oof)
    
    # save oof predictions
    oof.to_csv(OOF_PATH/f"oof-cv{repetition}.csv", index=False)
    # save models
    for fold,_model in enumerate(models):
        _model.save_model(ART_PATH/f"/model-cv{repetition}-fold{fold}.cbm", format="cbm")
          
    del _train, folds; gc.collect()

############################### repeated cross-validation step: 1/3 ################################


######################################## training model 1/5 ########################################


0:	learn: 0.6772284	test: 0.6808852	best: 0.6808852 (0)	total: 2.93s	remaining: 4h 3m 50s


20:	learn: 0.7320225	test: 0.7343627	best: 0.7343627 (20)	total: 10.1s	remaining: 40m 3s


40:	learn: 0.7375267	test: 0.7405049	best: 0.7405049 (40)	total: 17.4s	remaining: 35m 10s


...

4080:	learn: 0.8011730	test: 0.7886553	best: 0.7888217 (3820)	total: 24m 41s	remaining: 5m 33s


4100:	learn: 0.8012566	test: 0.7887657	best: 0.7888217 (3820)	total: 24m 48s	remaining: 5m 26s


4120:	learn: 0.8012713	test: 0.7887060	best: 0.7888217 (3820)	total: 24m 55s	remaining: 5m 19s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.7888216584
bestIteration = 3820

Shrink model to first 3821 iterations.


Training time: 126.87184566259384 min.
OOF recall_at4: 0.6582876089810482
OOF normalized_gini: 0.9214266240848463
OOF competition metric: 0.7898571165329473


CPU times: user 3d 1h 22min 55s, sys: 1h 21min 9s, total: 3d 2h 44min 5s
Wall time: 6h 5min 43s


In [13]:
results = list()

for oof in all_oof_dfs:  
    r = {
        "recall_at4": compute_recall_at4(oof.target.values, oof.pred.values),
        "gini": compute_normalized_gini(oof.target.values, oof.pred.values),
        "metric": compute_amex_metric(oof.target.values, oof.pred.values),
    }
    results.append(r)

results = pd.DataFrame(results)
display(results)

print("\nmean:")
display(results.mean(axis=0))

print("\nstd:")
display(results.std(axis=0))

,recall_at4,gini,metric
0,0.658271,0.921358,0.789814
1,0.658296,0.920889,0.789593
2,0.658288,0.921427,0.789857



mean:


recall_at4    0.658285
gini          0.921224
metric        0.789755
dtype: float64


std:


recall_at4    0.000013
gini          0.000292
metric        0.000142
dtype: float64

In [14]:
del train
gc.collect()

21

***
## make predictions and submit

In [15]:
def make_predictions(dataframe:pd.DataFrame, input_feats:list, models:list) -> np.array:
    preds = [
        model.predict(dataframe[input_feats], prediction_type="Probability")[:,1] 
        for model in models
    ]
    return np.mean(preds, axis=0)    

In [16]:
test = pd.read_parquet("../data/processed/dsv02/test.parquet")
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [17]:
%%time

all_preds = list()

for repetition in range(N_REPEATS):
    if "prediction" in sub.columns:
        sub.drop("prediction", axis=1, inplace=True)
    if "prediction" in test.columns:
        test.drop("prediction", axis=1, inplace=True)
        
    models = all_models[repetition]
    preds = make_predictions(test, input_feats, models)
    all_preds.append(preds)
       
    test["prediction"] = preds
    sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
    assert sub.prediction.isna().sum() == 0
    sub.to_csv(SUB_PATH/f"submission-cv{repetition}.csv", index=False)

CPU times: user 7min 28s, sys: 14.4 s, total: 7min 43s
Wall time: 1min 54s


In [18]:
%%time
# predict using all the trained models
if "prediction" in sub.columns:
    sub.drop("prediction", axis=1, inplace=True)
if "prediction" in test.columns:
    test.drop("prediction", axis=1, inplace=True)

test["prediction"] = np.mean(all_preds, axis=0)
sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
assert sub.prediction.isna().sum() == 0
sub.to_csv(SUB_PATH/f"submission-all.csv", index=False)

CPU times: user 3.49 s, sys: 836 ms, total: 4.33 s
Wall time: 4.63 s


***